In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [2]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord_u = 2
ord_phi = ord_u - 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord_u, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord_u, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord_phi) # fes phi

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
phi = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx)
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx 
        if ord_phi == 0:
            BF+= -delta * exp(psih) * vphi * dx -eps * (delta * vphi * dx)
        else:
            BF+= -delta * exp(psih) * vphi * dx -eps * (grad(delta) * grad(vphi) * dx)
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    






In [3]:
""" WORKING PARAMETERS FOR SIMPLEST BONDED TEST"""
### 1 PHI = -Z
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 25

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*40 ,softening_n))

### 2

In [4]:
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 20
        tol_newton = 1e-6
        tol_QN = 1e-8
        tol_delta = 1e-8
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_delta:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        print("Error",Integrate((uk-gfu.components[0])**2,mesh))

    Draw(gfu.components[0],mesh)



PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -8.416243757203388e-07
Solving nonlinear subproblem
Tol achieved -4.1076467252733906e-07
Solving nonlinear subproblem


KeyboardInterrupt: 

In [9]:
# DO VTK    
vtk = VTKOutput(ma=mesh,coefs=[gfu.components[0],gfu.components[1]],names=["u","phi"],filename="gel")

vtk.Do()

'gel'